# Scraping Instagram for the Master Thesis in MS Data Science & Marketing Analytics

### 1. Set-up the Chromedriver

In [1]:
#pip install bs4 selenium 
from selenium import webdriver
import time 

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--no-sandbox')
options.add_argument("--disable-setuid-sandbox")

### 2. Set-up working-directory and Import Tools/Functions

#### 2.1 Working-directory

In [2]:
import os

#### create nested folders
# from pathlib import Path
# Path("/NEW/").mkdir(parents=True, exist_ok=True)

#change wd
# Print the current working directory
print("Current working directory: {0}".format(os.getcwd()))

# Change the current working directory
os.chdir('C:/Users\lucas_0ey5q9s\Desktop\Master_thesis_data_git')

# Print the current working directory
print("Current working directory: {0}".format(os.getcwd()))
print(os.getcwd())


Current working directory: C:\Users\lucas_0ey5q9s\Desktop\Datasets\Uni
Current working directory: C:\Users\lucas_0ey5q9s\Desktop\Master_thesis_data_git
C:\Users\lucas_0ey5q9s\Desktop\Master_thesis_data_git


#### 2.2 Tools / Functions

In [3]:
#Import libraries
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
from itertools import accumulate
import re

#Define a function that checks for objects
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True
#Define a function to search for expression between defined strings!
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

### Open Lists

In [4]:
# Open ongoing list to safe the links!
LINKS=[]
f = open("Meta_links.txt", "r")
for x in f:
    LINKS.append(x.replace('\n',''))
    
influencer_list=[]
f = open("Final_Influencer_Collection.txt", "r")
for x in f:
    influencer_list.append(x.replace('\n',''))

### 3. Extract Random Influencers using Instagram's Tag-Search

#### Step 1: Get the all Links of Influencers shown in the Tag-Search  

In [271]:
# Get Random Influencer
driver = webdriver.Chrome()

# driver.get("https://www.instagram.com/accounts/login/")

# #Login
# driver.find_element_by_xpath("/html/body/div[2]/div/div/div/div[2]/button[1]").click()
# time.sleep(3)
# driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys("hagel559020")
# time.sleep(1)
# driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys("Schneefels0!")
# time.sleep(1)
# driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]').click()
# time.sleep(3)

random_links=[]

#Change tags to scrape!
## Tags used: 
##29-01
## 'giveaveeno', 'ethicalfashion','beautyblogger','partner','collaboration','cooperation',
## 'paidpartnership','sponsored', 'fashionblogger', 'mensfashion', 'womensfashion',
## 'ad','influencers','travel','advertisement','influencer', 'vacation',
## 'goodlife', 'communitylove','community'

# AVOID:
## Indiscriminative Hashtags perform worse:
## Travel, 

## NOT SO DAILY:
## 'revolvebeautybox','iweargarage','tommypartner','ipsypartner','ipsymakewaves',
## 'ipsytakeabreak','charleskeithcelebrates','imwithcharleskeith','ipsyglambagplus',
## 'desenio','showpoambassador','showpo','dollskill','camperlife','vanlife'
## 'tiuteam','cultgaia','curateur','walmartfashion','whitebikini','loungewear','blackbikini','ootd','ootn'

## 'instadaily','instagood','camperlife','vanlife','collaboration','cooperation','paidpartnership','sponsored',
## 'fashionblogger', 'mensfashion', 'womensfashion',
## 'ad','influencers','travel','advertisement','influencer',
## 'NYCblogger','NYCbloggers','LAblogger','LAbloggers', 'NYCbites','NYCfood',
## 'newyork','NYC','unitedstates','states', 'usa','michigan','roadtrip',
## 'california','usatravel','detroit','usatravels','florida',
## 'washington','tripatrips','run','newjersey','ohio','georgia','maryland','bhfyp'
tags=['instadaily','instagood','camperlife','vanlife','collaboration',
      'cooperation','paidpartnership','sponsored','fashionblogger', 'mensfashion', 
      'womensfashion','ad','influencers','travel','advertisement','influencer',
      'NYCblogger','NYCbloggers','LAblogger','LAbloggers', 'NYCbites','NYCfood',
      'newyork','NYC','unitedstates','states', 'usa','michigan','roadtrip',
      'california','usatravel','detroit','usatravels','florida',
      'washington','tripatrips','run','newjersey','ohio','georgia','maryland','bhfyp']

#locations=['https://www.instagram.com/explore/locations/212988663/new-york-new-york/']

for tag in tags:
    print(tag)
    driver.get('https://www.instagram.com/explore/tags/'+str(tag))
    time.sleep(5)
    
    SCROLL_PAUSE_TIME = 1

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        print("bottom of page")
        
    time.sleep(5)
    
    elements = driver.find_elements_by_xpath("//a[@href]")
    for elem in elements:
        urls = elem.get_attribute("href")
        if "p" in urls.split("/"):
            random_links.append(urls)
    time.sleep(5)
    random_links = list(set(random_links))
    print(len(random_links))

instadaily
0
instagood
0
camperlife


WebDriverException: Message: chrome not reachable
  (Session info: chrome=88.0.4324.104)


#### Step 2a: Given the Links of Step 1, append all links to an ongoing list that meet the following selection criteria:

1. Follower > 20k
2. Follower < 150k
3. Not in the ongoing list included!

In [270]:
### Open the existing ongoing list:
influencer_list=[]
f = open("Final_Influencer_Collection.txt", "r")
for x in f:
    influencer_list.append(x.replace('\n',''))
    
#Define range to search for follower
start ='edge_followed_by":{"count":'
end = '}'

# Check for received Links the Follower of each Influencer
driver = webdriver.Chrome()

driver.get("https://www.instagram.com/accounts/login/")

#Login
driver.find_element_by_xpath("/html/body/div[2]/div/div/div/div[2]/button[1]").click()
time.sleep(3)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys("hagel559020")
time.sleep(1)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys("Schneefels0!")
time.sleep(1)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]').click()
time.sleep(3)

t0= time.time()
counter = 0
for i in random_links:
    counter = counter + 1
    #go to tagsite
    driver.get(i)
    time.sleep(3)
    if check_exists_by_xpath('//*[@id="react-root"]/section/main/div/h2') == True:
        print('Ups... Page broken --> next')
    else:
        #Get Soup to check follower
        soup = BeautifulSoup(driver.page_source,'lxml')        
        follower = find_between(str(soup), start, end)
        follower = follower.replace(',', '')
        
        time.sleep(1)
        
        #Add influencer if follower between 20k - 100k
        if int(follower) > 20000 and int(follower) <= 350000:
            url = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/header/div[2]/div[1]/div[1]/span/a').get_attribute("href")
            if url in influencer_list:
                print("already added!")
            else: 
                influencer_list.append(url)
                print(follower,"! ",url, " appended!")
        else:
            print(follower, "does not suit in criteria...")
        
    print(counter, " / ",len(random_links), "done!")
    print(len(influencer_list)," have been collected now!")
    time.sleep(5)
        
# Save the list to use it later again
with open('Final_Influencer_Collection.txt', 'w') as f:
    for listitem in influencer_list:
        f.write('%s\n' % listitem)
f.close()

t1= time.time() - t0
print(t1)

1352 does not suit in criteria...
1  /  873 done!
4772  have been collected now!
753 does not suit in criteria...
2  /  873 done!
4772  have been collected now!
3467 does not suit in criteria...
3  /  873 done!
4772  have been collected now!
2125 does not suit in criteria...
4  /  873 done!
4772  have been collected now!
447 does not suit in criteria...
5  /  873 done!
4772  have been collected now!
2312 does not suit in criteria...
6  /  873 done!
4772  have been collected now!
already added!
7  /  873 done!
4772  have been collected now!
30 does not suit in criteria...
8  /  873 done!
4772  have been collected now!
3564 does not suit in criteria...
9  /  873 done!
4772  have been collected now!
27072 !  https://www.instagram.com/vienna.gamanacasa/  appended!
10  /  873 done!
4773  have been collected now!
14630 does not suit in criteria...
11  /  873 done!
4773  have been collected now!
359 does not suit in criteria...
12  /  873 done!
4773  have been collected now!
2479 does not sui

ValueError: invalid literal for int() with base 10: ''

#### Step 2b: Just in case the code from step 2a was crashed/interrupted: Save List

In [274]:
with open('Final_Influencer_Collection.txt', 'w') as f:
    for listitem in influencer_list:
        f.write('%s\n' % listitem)
f.close()

In [277]:
len(influencer_list)

5347

#### EXTRA: If some Influencers have been extracted manually, please run this code to append manually found influencer to the Final_Influencer_Collection if they are not already included

In [276]:
#Copy new file into list_manual
list_manual=[]
f = open("Manually_extracted_Influencer.txt", "r")
for x in f:
    list_manual.append(x.replace('\n',''))

#Copy Final_Influencer_Collection into influencer_list
influencer_list=[]
f = open("Final_Influencer_Collection.txt", "r")
for x in f:
    influencer_list.append(x.replace('\n',''))
    
######################################################################
#Put each element of list_manual in url format and append to the initial list if it is not included in the list so far
count = 0
for i in list_manual:
    count = count + 1
    link = "https://www.instagram.com/"+i+"/" 
    if link in influencer_list:
        print("already added!")
    else: 
        influencer_list.append(link)
        print(count,": ", link, " appended!")
######################################################################

# Save the list to use it later again
with open('Final_Influencer_Collection.txt', 'w') as f:
    for listitem in influencer_list:
        f.write('%s\n' % listitem)
f.close()

1 :  https://www.instagram.com/natalieinthecity_/  appended!
2 :  https://www.instagram.com/ajetsetjournal/  appended!
already added!
already added!
5 :  https://www.instagram.com/ewdatsgross/  appended!
6 :  https://www.instagram.com/everydaywithhk/  appended!
7 :  https://www.instagram.com/suhedanur/  appended!
already added!
9 :  https://www.instagram.com/alittledoseofjen/  appended!
10 :  https://www.instagram.com/olia_chicago/  appended!
11 :  https://www.instagram.com/karenchaiane/  appended!
12 :  https://www.instagram.com/nikarichii/  appended!
13 :  https://www.instagram.com/thebecka.blog/  appended!
14 :  https://www.instagram.com/eriikalima/  appended!
15 :  https://www.instagram.com/leehocampo/  appended!
16 :  https://www.instagram.com/_ikiaaa_/  appended!
17 :  https://www.instagram.com/chicago.wine.dude/  appended!
18 :  https://www.instagram.com/elainerau/  appended!
19 :  https://www.instagram.com/irin_dudkina/  appended!
20 :  https://www.instagram.com/maddyreviews/  

756 :  https://www.instagram.com/moses.daniel.matthew.735/  appended!
757 :  https://www.instagram.com/kathrynpackard/  appended!
758 :  https://www.instagram.com/bluecavi/  appended!


### 4. Start scraping Meta Information + individual Posting Links

In [286]:
# Open ongoing list to safe the links!
LINKS=[]
f = open("Meta_links.txt", "r")
for x in f:
    LINKS.append(x.replace('\n',''))
driver = webdriver.Chrome()

driver.get("https://www.instagram.com/accounts/login/")

#Login
#driver.find_element_by_xpath("/html/body/div[2]/div/div/div/div[2]/button[1]").click()
time.sleep(3)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys("alexhekarts")
time.sleep(1)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys("Schneefels0!")
time.sleep(1)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]').click()
time.sleep(3)

#create empty lists
Page_links = []
Name = []
Meta_descr = []
Total_posts = []
Follower = []
Following = []
Verified = []


#Loop through influencer
### last time: 4500:5001
counter = 4500
for influencer in influencer_list[4500:5001]:
    
    #Get Page
    driver.get(influencer)
    counter = counter + 1
    time.sleep(2)
    if check_exists_by_xpath('//*[@id="react-root"]/section/main/div/h2') == True:
        print("Page is broken!")
        
    else:
        if check_exists_by_xpath('/html/body/div/div[1]/div/div/h2') == True:
            print("Page is broken!")
        else: 
            if check_exists_by_xpath('//*[@id="react-root"]/section/main/div/div/article/div[1]/div/h2') == True:
                print("This account is private!")
            else:
                #Extract Meta-Information
                ## Store Link
                Page_links.append(influencer)
    
                ## Name
                if check_exists_by_xpath('//*[@id="react-root"]/section/main/div/header/section/div[1]/h2') == True:
                    element = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/div[1]/h2')
                    Name.append(element.text)
                    time.sleep(0.5)
                else:
                    element = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/div[1]/h1')
                    Name.append(element.text)
                    time.sleep(0.5)
        
                ## Meta-description
                if check_exists_by_xpath('//*[@id="react-root"]/section/main/div/header/section/div[2]/span') == True:
                    element = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/div[2]/span')
                    Meta_descr.append(element.text)
                    time.sleep(0.5)
                else:
                    Meta_descr.append("NA")
    
                ## Total Posts
                element = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/ul/li[1]/span/span')
                Total_posts.append(element.text)
                time.sleep(0.5)
    
                ## Follower
                element = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/ul/li[2]/a/span')
                Follower.append(element.text)
                time.sleep(0.5)
    
                ## Following
                if check_exists_by_xpath('//*[@id="react-root"]/section/main/div/header/section/ul/li[3]/a/span') == True:
                    element = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/ul/li[3]/a/span')
                    Following.append(element.text)
                    time.sleep(1)
                else:
                    element = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/ul/li[3]/span/span')
                    Following.append(element.text)
                    time.sleep(1)
    
                ## Check if verified user
                if check_exists_by_xpath('//*[@id="react-root"]/section/main/div/header/section/div[1]/div[1]/span') == True:
                    v = 1
                else:
                    v = 0
                # append to list
                Verified.append(v)
    
                time.sleep(1)
    
                extract = []
                elements = driver.find_elements_by_xpath("//a[@href]")
                for elem in elements:
                    urls = elem.get_attribute("href")
                    if "p" in urls.split("/"):
                        extract.append(urls)
                LINKS = LINKS + extract[1:11]
                print("Influencer: ", counter," done!")
    
# Save the list to use it later again
with open('Meta_links.txt', 'w') as f:
    for listitem in LINKS:
        f.write('%s\n' % listitem)
f.close()

print("List is closed and safed!")

Influencer:  4501  done!
Influencer:  4502  done!
Influencer:  4503  done!
Influencer:  4504  done!
Influencer:  4505  done!
Influencer:  4506  done!
Influencer:  4507  done!
Influencer:  4508  done!
Influencer:  4509  done!
Influencer:  4510  done!
Influencer:  4511  done!
Influencer:  4512  done!
Influencer:  4513  done!
Influencer:  4514  done!
Influencer:  4515  done!
Influencer:  4516  done!
Influencer:  4517  done!
Influencer:  4518  done!
Influencer:  4519  done!
Influencer:  4520  done!
Influencer:  4521  done!
Influencer:  4522  done!
Influencer:  4523  done!
Influencer:  4524  done!
Influencer:  4525  done!
Influencer:  4526  done!
Influencer:  4527  done!
Influencer:  4528  done!
Influencer:  4529  done!
Influencer:  4530  done!
Influencer:  4531  done!
Influencer:  4532  done!
Influencer:  4533  done!
Influencer:  4534  done!
Influencer:  4535  done!
Influencer:  4536  done!
Influencer:  4537  done!
Influencer:  4538  done!
Influencer:  4539  done!
Influencer:  4540  done!


Influencer:  4831  done!
Influencer:  4832  done!
Influencer:  4833  done!
Influencer:  4834  done!
Influencer:  4835  done!
Influencer:  4836  done!
Influencer:  4837  done!
Influencer:  4838  done!
Influencer:  4839  done!
Influencer:  4840  done!
Influencer:  4841  done!
Influencer:  4842  done!
Influencer:  4843  done!
Influencer:  4844  done!
Influencer:  4845  done!
Influencer:  4846  done!
Influencer:  4847  done!
Influencer:  4848  done!
Influencer:  4849  done!
Influencer:  4850  done!
Influencer:  4851  done!
Influencer:  4852  done!
Influencer:  4853  done!
Influencer:  4854  done!
Influencer:  4855  done!
Influencer:  4856  done!
Influencer:  4857  done!
Influencer:  4858  done!
Influencer:  4859  done!
Influencer:  4860  done!
Influencer:  4861  done!
Influencer:  4862  done!
Influencer:  4863  done!
Influencer:  4864  done!
Page is broken!
Page is broken!
Influencer:  4867  done!
Page is broken!
Influencer:  4869  done!
Influencer:  4870  done!
Influencer:  4871  done!
In

In [9]:
influencer_list=[]
f = open("Final_Influencer_Collection.txt", "r")
for x in f:
    influencer_list.append(x.replace('\n',''))
    

In [10]:
len(influencer_list)

5347

### Save Posting LINKS from Influencers

In [18]:
# Open ongoing list to safe the links!
LINKS=[]
f = open("Meta_links.txt", "r")
for x in f:
    LINKS.append(x.replace('\n',''))
driver = webdriver.Chrome()

driver.get("https://www.instagram.com/accounts/login/")

#Login
#driver.find_element_by_xpath("/html/body/div[2]/div/div/div/div[2]/button[1]").click()
time.sleep(3)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys("cedrikzietlow")
time.sleep(1)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys("Schneefels0!")
time.sleep(1)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]').click()
time.sleep(3)


#Loop through influencer
### last time: 3001-5001
counter = 3000
for influencer in influencer_list[3001:5001]:
    
    #Get Page
    driver.get(influencer)
    counter = counter + 1
    time.sleep(1)
    if check_exists_by_xpath('//*[@id="react-root"]/section/main/div/h2') == True:
        print("Page is broken!")
    else:
        if check_exists_by_xpath('/html/body/div/div[1]/div/div/h2') == True:
            print("Page is broken!")
        else: 
            if check_exists_by_xpath('//*[@id="react-root"]/section/main/div/div/article/div[1]/div/h2') == True:
                print("This account is private!")
            else:
                extract = []
                elements = driver.find_elements_by_xpath("//a[@href]")
                for elem in elements:
                    urls = elem.get_attribute("href")
                    if "p" in urls.split("/"):
                        extract.append(urls)
                LINKS = LINKS + extract[1:11]
                print("Influencer: ", counter," done!")
                
                time.sleep(3)

# Save the list to use it later again
with open('Meta_links.txt', 'w') as f:
    for listitem in LINKS:
        f.write('%s\n' % listitem)
f.close()

print("List is closed and safed!")

Influencer:  3001  done!
Influencer:  3002  done!
Influencer:  3003  done!
Influencer:  3004  done!
Influencer:  3005  done!
Influencer:  3006  done!
Influencer:  3007  done!
Influencer:  3008  done!
Influencer:  3009  done!
Influencer:  3010  done!
Influencer:  3011  done!
Influencer:  3012  done!
Influencer:  3013  done!
Influencer:  3014  done!
Influencer:  3015  done!
Influencer:  3016  done!
Influencer:  3017  done!
Influencer:  3018  done!
Influencer:  3019  done!
Influencer:  3020  done!
Influencer:  3021  done!
Influencer:  3022  done!
Influencer:  3023  done!
Influencer:  3024  done!
Influencer:  3025  done!
Influencer:  3026  done!
Influencer:  3027  done!
Influencer:  3028  done!
Influencer:  3029  done!
Influencer:  3030  done!
Influencer:  3031  done!
Influencer:  3032  done!
Influencer:  3033  done!
Influencer:  3034  done!
Influencer:  3035  done!
Influencer:  3036  done!
Influencer:  3037  done!
Page is broken!
Influencer:  3039  done!
Influencer:  3040  done!
Influence

Influencer:  3330  done!
Influencer:  3331  done!
Influencer:  3332  done!
Influencer:  3333  done!
Influencer:  3334  done!
Influencer:  3335  done!
Influencer:  3336  done!
Influencer:  3337  done!
Influencer:  3338  done!
Influencer:  3339  done!
Influencer:  3340  done!
Influencer:  3341  done!
Influencer:  3342  done!
Influencer:  3343  done!
Influencer:  3344  done!
Influencer:  3345  done!
Influencer:  3346  done!
Influencer:  3347  done!
Influencer:  3348  done!
Influencer:  3349  done!
Influencer:  3350  done!
Influencer:  3351  done!
Influencer:  3352  done!
Influencer:  3353  done!
Influencer:  3354  done!
Influencer:  3355  done!
Influencer:  3356  done!
Influencer:  3357  done!
Influencer:  3358  done!
Influencer:  3359  done!
Influencer:  3360  done!
Influencer:  3361  done!
Influencer:  3362  done!
Influencer:  3363  done!
Influencer:  3364  done!
Influencer:  3365  done!
Influencer:  3366  done!
Influencer:  3367  done!
Influencer:  3368  done!
Influencer:  3369  done!


Influencer:  3662  done!
Influencer:  3663  done!
Influencer:  3664  done!
Influencer:  3665  done!
Influencer:  3666  done!
Influencer:  3667  done!
Influencer:  3668  done!
Influencer:  3669  done!
Influencer:  3670  done!
Influencer:  3671  done!
This account is private!
Influencer:  3673  done!
Influencer:  3674  done!
Influencer:  3675  done!
Page is broken!
Influencer:  3677  done!
Influencer:  3678  done!
Influencer:  3679  done!
Influencer:  3680  done!
Influencer:  3681  done!
Influencer:  3682  done!
Influencer:  3683  done!
Influencer:  3684  done!
Influencer:  3685  done!
Influencer:  3686  done!
Influencer:  3687  done!
Influencer:  3688  done!
Influencer:  3689  done!
Influencer:  3690  done!
Influencer:  3691  done!
Influencer:  3692  done!
Influencer:  3693  done!
Influencer:  3694  done!
Influencer:  3695  done!
Influencer:  3696  done!
Influencer:  3697  done!
Influencer:  3698  done!
Influencer:  3699  done!
Influencer:  3700  done!
Influencer:  3701  done!
Influence

Influencer:  4001  done!
Influencer:  4002  done!
Influencer:  4003  done!
Influencer:  4004  done!
Influencer:  4005  done!
Influencer:  4006  done!
Influencer:  4007  done!
Influencer:  4008  done!
Influencer:  4009  done!
Influencer:  4010  done!
This account is private!
Influencer:  4012  done!
Influencer:  4013  done!
Influencer:  4014  done!
Influencer:  4015  done!
Influencer:  4016  done!
Influencer:  4017  done!
Influencer:  4018  done!
Influencer:  4019  done!
Influencer:  4020  done!
Influencer:  4021  done!
Influencer:  4022  done!
Influencer:  4023  done!
Influencer:  4024  done!
Influencer:  4025  done!
Influencer:  4026  done!
Influencer:  4027  done!
Influencer:  4028  done!
This account is private!
Influencer:  4030  done!
Influencer:  4031  done!
Page is broken!
Influencer:  4033  done!
Influencer:  4034  done!
Influencer:  4035  done!
Influencer:  4036  done!
Influencer:  4037  done!
Influencer:  4038  done!
Influencer:  4039  done!
Page is broken!
Influencer:  4041 

Influencer:  4340  done!
Influencer:  4341  done!
Influencer:  4342  done!
Influencer:  4343  done!
Influencer:  4344  done!
Influencer:  4345  done!
Influencer:  4346  done!
Influencer:  4347  done!
Page is broken!
Influencer:  4349  done!
Influencer:  4350  done!
Page is broken!
Influencer:  4352  done!
Influencer:  4353  done!
Influencer:  4354  done!
Page is broken!
Influencer:  4356  done!
Influencer:  4357  done!
Influencer:  4358  done!
Influencer:  4359  done!
Page is broken!
Influencer:  4361  done!
Influencer:  4362  done!
Influencer:  4363  done!
Influencer:  4364  done!
Influencer:  4365  done!
Influencer:  4366  done!
Influencer:  4367  done!
Page is broken!
Influencer:  4369  done!
Influencer:  4370  done!
Influencer:  4371  done!
Influencer:  4372  done!
Influencer:  4373  done!
Influencer:  4374  done!
Influencer:  4375  done!
Influencer:  4376  done!
Influencer:  4377  done!
Influencer:  4378  done!
Influencer:  4379  done!
Influencer:  4380  done!
Influencer:  4381  d

Influencer:  4671  done!
Influencer:  4672  done!
Influencer:  4673  done!
Influencer:  4674  done!
Influencer:  4675  done!
Influencer:  4676  done!
Influencer:  4677  done!
Influencer:  4678  done!
Influencer:  4679  done!
Influencer:  4680  done!
Influencer:  4681  done!
Influencer:  4682  done!
Influencer:  4683  done!
Influencer:  4684  done!
Influencer:  4685  done!
Influencer:  4686  done!
Influencer:  4687  done!
Influencer:  4688  done!
Influencer:  4689  done!
Influencer:  4690  done!
Influencer:  4691  done!
Influencer:  4692  done!
Influencer:  4693  done!
Influencer:  4694  done!
Influencer:  4695  done!
Influencer:  4696  done!
Influencer:  4697  done!
Influencer:  4698  done!
Influencer:  4699  done!
Influencer:  4700  done!
Influencer:  4701  done!
Influencer:  4702  done!
Influencer:  4703  done!
Influencer:  4704  done!
Influencer:  4705  done!
Influencer:  4706  done!
Influencer:  4707  done!
Influencer:  4708  done!
Influencer:  4709  done!
Influencer:  4710  done!


#### Step 4b: Just in case the code from step 4 was crashed/interrupted: Save List

In [6]:
# Save the list to use it later again
with open('Meta_links.txt', 'w') as f:
    for listitem in LINKS:
        f.write('%s\n' % listitem)
f.close()

In [19]:
len(LINKS)

46867

In [289]:
print(
    len(Name),
    len(Total_posts),
    len(Follower),
    len(Following),
    len(Verified),
    len(Meta_descr),
    len(Page_links)
)

487 487 487 487 487 487 487


In [294]:
len(influencer_list)

5347

In [291]:
# Change Name according to the used list!!!
Influencer_meta_4500_5001 = {'Name':Name, 
                      'Total_posts':Total_posts, 
                      'Follower':Follower, 
                      'Following':Following, 
                      'Verified':Verified, 
                      'Meta_descr':Meta_descr,
                      'Page_links':Page_links}
Influencer_meta_4500_5001 = pd.DataFrame(Influencer_meta_4500_5001)
Influencer_meta_4500_5001['Meta_descr'] = Influencer_meta_4500_5001['Meta_descr'].replace('\n',' ', regex=True)
Influencer_meta_4500_5001.to_pickle("./Meta_data/Influencer_meta_4500_5001.pkl")

In [45]:
# ## Open Pickle data!
# import pickle
# opened_again = pickle.load(open( "Influencer_meta_1_90.pkl", "rb" ))

,Name,Total_posts,Follower,Following,Verified,Meta_descr,Page_links
0,actor_avinash_official,537,39k,204,0,"Actor, Model & Dancer Fashion Enthusiast ♦️Opt...",https://www.instagram.com/actor_avinash_official/
1,africangallerybay,666,31.8k,20,0,Go join our youtube team now 👇🏽👇🏽,https://www.instagram.com/africangallerybay/
2,arianerein,143,21.3k,"1,704",0,⛅ hey there! it's ariane 📩 arianerein.collab@g...,https://www.instagram.com/arianerein/
3,andreea__87,"1,069",26.7k,"3,229",0,#naturelovers 🌿#fashion 👒#beauty💄 -DM for coll...,https://www.instagram.com/andreea__87/
4,rayuaniklan,"1,174",38.7k,394,0,Menikmati kreativitas iklan di indonesia. kont...,https://www.instagram.com/rayuaniklan/
...,...,...,...,...,...,...,...
85,birdandkite,"2,769",66.4k,"2,180",0,Bali to Byron Bay ~ Conscious & Slow - Artisan...,https://www.instagram.com/birdandkite/
86,klarstein_de,"1,088",45.2k,988,0,"We ❤️ Food! Gesund, lecker, vielfältig! Lasst ...",https://www.instagram.com/klarstein_de/
87,itsmarcelloofficial,820,21k,322,0,Nordstrom Customer Service All-Star Interactiv...,https://www.instagram.com/itsmarcelloofficial/
88,julienfranks,687,87.1k,447,0,I’m Julie ✌️Sharing affordable finds. ✨Click t...,https://www.instagram.com/julienfranks/


### 5. Start scraping individual posting links + Comment Section

In [ ]:
# Open ongoing list to safe the links!
LINKS=[]
f = open("Meta_links.txt", "r")
for x in f:
    LINKS.append(x.replace('\n',''))

driver = webdriver.Chrome()

driver.get("https://www.instagram.com/accounts/login/")

#Login
#driver.find_element_by_xpath("/html/body/div[2]/div/div/div/div[2]/button[1]").click()
time.sleep(3)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys("gustavseil")
time.sleep(1)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys("Schneefels0!")
time.sleep(1)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]').click()
time.sleep(3)

#create empty lists
image_sources = []
description = []
likes = []
comments = []
video = []
name = []
paid_partner = []
locations = []
reply_count = []
reply_sum = []
timeprint = []
sequence = []
tagged = []

time_vector = []

############# CHANGE ############
#oldstart = 14045
#oldend = 15000
start_i = 25840
end = 27000
#################################

start_n = start_i + 1
#Dataframe name
range_name = '_'+str(start_n)+'_'+str(end)

#Dict to save dataframes!
df = {}

#Reset the counter
post_link = []
post_nr = []
comment_counter_by_post = []
counter_posts_add = []

#Loop through posts
#Track Time for whole loop
t0= time.time()
counter_posts = start_i

for link in LINKS[start_i:end]:
    
    counter_comments = []
    name_comments = []
    counter_posts = counter_posts + 1
    
    # get ith post
    driver.get(link)
    time.sleep(2)
    
    if check_exists_by_xpath('//*[@id="react-root"]/section/main/div/h2') == True:
        print("Page is broken!")
        
    elif check_exists_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/h2') == True:
        print('Video not available')
    else:
        soup = BeautifulSoup(driver.page_source,'lxml') 
        #Track Time for comments
        t1 = time.time()
        post_nr.append(counter_posts)
        time.sleep(1)
        #get image
        try:
            driver.find_element_by_class_name('tWeCl').get_attribute('type') == "video/mp4"
            print("let me check if its a video....")
        except:
            print("Seems to be an image...")
            start = '"display_url":"'
            end = '","display_resources"'
            #Get Soup to check follower       
            source_dirty = find_between(str(soup), start, end)
            source = source_dirty.replace('\\u0026','&')
            time.sleep(2)
            image_sources.append(source)
            #safe image
            file_location = os.path.join(os.getcwd()+'\Pictures', link[28:-1]+".jpg")
            urllib.request.urlretrieve(source, file_location)
            v = 0
            #get likes
            if check_exists_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/section[2]/div/div/a/span') == True:
                l = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/section[2]/div/div/a/span')
                likes.append(l.text)
            else:
                likes.append('0')
            
        else:
            image_sources.append('video')
            source = 'video'
            v = 1
            #get views
            try:
                views = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/section[2]/div/span/span')
            except:
                views = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/section[2]/div/div/a/span')
            likes.append(views.text)
            print("video instead of image appended")
        
        video.append(v)
    
        #get instagram name
        if check_exists_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/header/div[2]/div[1]/div[1]/span/a') == True:
            n = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/header/div[2]/div[1]/div[1]/span/a')
            name.append(n.text)
        else:
            n = driver.find_element_by_xpath('/html/body/div[4]/div[2]/div/article/header/div[2]/div[1]/div[1]/span/a')
            name.append(n.text)
        
        #Get Post Link "name"
        post_link.append(link[28:-1])
        
        #Get timeprint
        t = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/div[2]/a/time').get_attribute('datetime')
        timeprint.append(t)
        
        #get image description
        if check_exists_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/div[1]/ul/div/li/div/div/div[2]/span') == True:
            d = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/div[1]/ul/div/li/div/div/div[2]/span')
            d = d.get_attribute("innerHTML")
        else:
            d = 'NA'
        
        description.append(d)
    
        #get paid_partneship
        if check_exists_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/header/div[2]/div[2]/div[1]/span/a') == True:
            paid = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/header/div[2]/div[2]/div[1]/span/a')
            paid_partner.append(paid.text)
        else:
            paid_partner.append('NA')
    
        #get Location
        if check_exists_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/header/div[2]/div[2]/div[2]/a') == True:
            loc = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/header/div[2]/div[2]/div[2]/a')
            locations.append(loc.text)
        else:
            locations.append('NA')
    
        time.sleep(1)
    
        #Get #comments
        start='"edge_media_to_parent_comment":{"count":'
        end=',"'
        try:
            c = find_between(str(soup), start, end)
            comments.append(c)
        except:
            comments.append('NA')
        
        time.sleep(1)
        #Get tags
        tags = []
        r_tags = soup.find_all('span',attrs={'class':'eg3Fv'})
        soup_2_tags = BeautifulSoup(str(r_tags),'lxml') 
        spans_tags = soup_2_tags.find_all('span')
        tags = [i.text.strip() for i in spans_tags if i != '']
        tagged.append(str(tags).strip('[]'))
        
        time.sleep(1)
        
        #Check if this is a sequence
        if check_exists_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[2]/div/div[1]/div[2]/div/button/div') == True:
            sequence.append('1')
        else:
            sequence.append('0')
        
        time.sleep(2)
        #Print time for comments
        t1_1 = time.time() - t1
        print(source)
        print("Scraping time for post", counter_posts, ": ", t1_1, 'LINK:', link)
        time_vector.append(round(t1_1,1))
        
        print('---------------------------------------------------------')
    
#Print time for whole loop
t0_1 = time.time() - t0
print("")
print(">> Time whole loop: ", t0_1)
print(">> All iterations done BUT NOT SAVED!")
end_index = end - start_i
start_n = start_i + 1

Page is broken!
Page is broken!
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/145624557_459650191717107_1934398507509541611_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=103&_nc_ohc=2bAkljstZDMAX9QXGPj&tp=1&oh=fa79fd24bed9f9c7baa415d4144dfb03&oe=6059D4DE
Scraping time for post 25843 :  10.781448364257812 LINK: https://www.instagram.com/p/CK5OX-5A2rB/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/146294848_522037125450328_5164617287493980927_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=104&_nc_ohc=XI2wQg2kH1IAX8nFgy1&tp=1&oh=5003d109a04aa23bb06f773c2b0c7c13&oe=605B7288
Scraping time for post 25844 :  10.709514856338501 LINK: https://www.instagram.com/p/CK4s4ErA-Dy/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/145850070_213293927140385_

Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/145620735_434766434386489_5007759257938739240_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=101&_nc_ohc=--W4Uws3iBAAX8Svpnz&tp=1&oh=4ff29358a7378ffc7b830b23bfdb5342&oe=605A10A8
Scraping time for post 25864 :  9.03656268119812 LINK: https://www.instagram.com/p/CK4tjwdJ7JV/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/145182602_1139565786474632_5962846707477755835_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=108&_nc_ohc=dPuWYgxaEK4AX-jzisu&tp=1&oh=a6377733a1d20f491ee9e7104d952e6a&oe=6059C656
Scraping time for post 25865 :  11.046261548995972 LINK: https://www.instagram.com/p/CKzFafsJVfu/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/145025194_1591600464363955_6900032369

Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/141847887_119092646748343_8845225610397854149_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=107&_nc_ohc=6DVJL4Aq5Y4AX_1Z4Oz&tp=1&oh=fcc55a2c5f59aea219c02631b4db63a8&oe=605A5FF7
Scraping time for post 25884 :  9.78660774230957 LINK: https://www.instagram.com/p/CKdfWxjnDyk/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/139594954_241692547408867_405299782206783381_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=107&_nc_ohc=0A0gpmptwwUAX_18Dzh&tp=1&oh=501461b29aa20a8e1269c94b45c503e5&oe=605A3D99
Scraping time for post 25885 :  9.45357370376587 LINK: https://www.instagram.com/p/CKLmeIIHS9N/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/139262667_236815434632372_262135386680512

Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/146004862_867319660727006_4703031874334841016_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=102&_nc_ohc=nde2G1t_MJ0AX9Lkzms&tp=1&oh=b7feaa4b956f521bcc80f8d608726e9a&oe=605A0A75
Scraping time for post 25904 :  9.675411701202393 LINK: https://www.instagram.com/p/CK63hr5FX10/
---------------------------------------------------------
let me check if its a video....
video instead of image appended
video
Scraping time for post 25905 :  6.155659198760986 LINK: https://www.instagram.com/p/CK4OwgIDfZG/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/145964932_1084417268667447_2648454620607554222_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=103&_nc_ohc=xkcMLwJ_vaoAX_Jov5b&tp=1&oh=cd907f003ed1f6046fb3e7aaae57ba81&oe=605A991A
Scraping time for post 25906 :  9.99011492729187 LINK: https://www.instagram.com/p/

Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/145971645_1055096658320245_1539275014812540659_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=101&_nc_ohc=6RnduTw5QmIAX_1B2fk&tp=1&oh=d3e47d7e9ad8c7aefeb1120ea811c268&oe=6058E4AB
Scraping time for post 25926 :  9.972879409790039 LINK: https://www.instagram.com/p/CK3YIOMApFn/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/144607791_1314104192301023_1699677834885588148_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=111&_nc_ohc=AL6x-KYT6rcAX9MvEzQ&tp=1&oh=e09a5dc0df693b021c8b758727ae8f7e&oe=605ABD63
Scraping time for post 25927 :  8.665126085281372 LINK: https://www.instagram.com/p/CK1PXmDpugj/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/145250160_162150852166896_5656159917635647308_n.jpg?_nc_ht=scontent-b

Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/139427890_707313906598659_8243105430711759677_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=103&_nc_ohc=CFhCbA5_vFAAX8o8VzX&tp=1&oh=75a5d493d45c24c4c2bce888fb2bc744&oe=6058ABEA
Scraping time for post 25946 :  10.250452518463135 LINK: https://www.instagram.com/p/CKG1TX_l422/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/137238099_474654096863172_362005168293369362_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=100&_nc_ohc=bdHxfVt-2IQAX9O9sFu&tp=1&oh=1ebc883a7b6d95590b0d8b71d3481d8a&oe=605A6831
Scraping time for post 25947 :  9.278092384338379 LINK: https://www.instagram.com/p/CJ8k0ppFvrp/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/136377768_1900011516818194_13267446557

Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/146773231_330553608294250_3626030662789621047_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=110&_nc_ohc=QKvLhHBpVOcAX8Pv3sD&tp=1&oh=ba5f8c2a5720af5d24dfd7a6d5ce79b1&oe=605B3B4E
Scraping time for post 25969 :  9.314278364181519 LINK: https://www.instagram.com/p/CLChX0nAzeZ/
---------------------------------------------------------
let me check if its a video....
video instead of image appended
video
Scraping time for post 25970 :  6.17092490196228 LINK: https://www.instagram.com/p/CK4HZrngRCF/
---------------------------------------------------------
let me check if its a video....
video instead of image appended
video
Scraping time for post 25971 :  6.191680669784546 LINK: https://www.instagram.com/p/CKzSC5Pg8Np/
---------------------------------------------------------
let me check if its a video....
video instead of image appended
video
Scraping time for post 25972 :  6.2099602222442

let me check if its a video....
video instead of image appended
video
Scraping time for post 25993 :  6.183333873748779 LINK: https://www.instagram.com/p/CKkP_oXjPN-/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/140286213_1046623836099319_6079272547237333228_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=107&_nc_ohc=De8atrqaaz0AX-OpGCD&tp=1&oh=fc2b8f9b1d5aff44f26206ffaf8d54ed&oe=605A11A0
Scraping time for post 25994 :  10.198965787887573 LINK: https://www.instagram.com/p/CKP9NUqjhYq/
---------------------------------------------------------
let me check if its a video....
video instead of image appended
video
Scraping time for post 25995 :  6.195065498352051 LINK: https://www.instagram.com/p/CKKdBJPjlCi/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/139026063_1215905622

let me check if its a video....
video instead of image appended
video
Scraping time for post 26018 :  6.19249153137207 LINK: https://www.instagram.com/p/CK_F2DNHr0o/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/147126184_765702347386292_4081715943549506513_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=108&_nc_ohc=Dy2bFAMH5_kAX8NZo8X&tp=1&oh=4bff382670a20f39dbc873f105b5f2c8&oe=60594D43
Scraping time for post 26019 :  19.84729027748108 LINK: https://www.instagram.com/p/CLDuPVrJSOp/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/147087833_1136448586783698_4102098250560291007_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=106&_nc_ohc=2v55yN5q9SgAX8LIrIy&tp=1&oh=8cc397b25647af88c5f9c91147d6e7a1&oe=605A9BA5
Scraping time for post 26020 :  12.107720136642456 LINK: https

Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/134556355_1508107276246330_8527789200797531578_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=110&_nc_ohc=EQAoy_y7WHoAX_6vSR1&tp=1&oh=978375e477686d55876c8010d2bcf751&oe=605A1B69
Scraping time for post 26038 :  10.074377059936523 LINK: https://www.instagram.com/p/CJfXktgloTo/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/145942561_644704649577971_2579612546680405302_n.jpg?_nc_ht=scontent-bos3-1.cdninstagram.com&_nc_cat=111&_nc_ohc=e10-NFFvsBIAX9hsKyH&tp=1&oh=e3b82eb102a0a2b1c7386f1f3ea20b63&oe=60590DC1
Scraping time for post 26039 :  12.22515082359314 LINK: https://www.instagram.com/p/CK31HinsJK-/
---------------------------------------------------------
Seems to be an image...
https://scontent-bos3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/145091870_3754523207927288_231765277

In [6]:
print(len(name))
print(len(post_link))
print(len(post_nr))
print(len(paid_partner))
print(len(locations))
print(len(video))
print(len(comments))
print(len(likes))
print(len(description))
print(len(image_sources))
print(len(timeprint))
print(len(sequence))
print(len(tagged))

1059
1059
1060
1059
1059
1059
1059
1059
1059
1060
1059
1059
1059


### save Posts

In [7]:
############# CHANGE ############
start_i = 24766
end = 25825
#################################

end_index = end - start_i
start_n = start_i + 1
#Dataframe name
range_name = '_'+str(start_n)+'_'+str(end)

In [8]:
# Change Name according to the used list!!!
df['posts_df'+range_name] = {'name':name[:end_index],
                             'link_name':post_link[:end_index],
                             '#post':post_nr[:end_index],
                             'paid_partner':paid_partner[:end_index],
                             'location': locations[:end_index],
                             'is_video':video[:end_index], 
                             '#comments':comments[:end_index],
                             '#likes':likes[:end_index], 
                             'description':description[:end_index], 
                             'image_source':image_sources[:end_index],
                             'timeprint':timeprint[:end_index],
                             'image_sequence':sequence[:end_index],
                             'tagged':tagged[:end_index]}
df['posts_df'+range_name] = pd.DataFrame(df['posts_df'+range_name])
df['posts_df'+range_name].to_pickle("./Posts_data/"+str('posts_df'+range_name)+".pkl")

print(df['posts_df'+range_name])


                 name    link_name  #post paid_partner  \
0          wandergasm  CI3uzLEqMIw  24767           NA   
1          wandergasm  CIynszVgbJC  24768           NA   
2          wandergasm  CIjDZK5nvE-  24769           NA   
3          wandergasm  CIgh22vAWAc  24770        nobis   
4          wandergasm  CIbefKIK-Ky  24771           NA   
...               ...          ...    ...          ...   
1054  necklaceofpearl  CJ8u1o-sDVK  25836           NA   
1055  necklaceofpearl  CJ5mfX3MKp_  25837           NA   
1056  necklaceofpearl  CJtW0Qfszuh  25838           NA   
1057      babygargoyl  CLAb2s5gAue  25839           NA   
1058      babygargoyl  CK8EJ3Fg29A  25840           NA   

                        location  is_video #comments  #likes  \
0                             NA         1        12     247   
1     Saint-Martin-de-Belleville         0        24     874   
2                    Val Thorens         0        15     817   
3            Rhône-Alpes, France         0     

### save Comments

In [109]:
df['final_df_comments_1_1000'].to_pickle("./Comments_data/"+str('final_df_comments'+range_name)+".pkl")

In [136]:
df

{'final_df_comments_84_500':                       name    link_name #post comment_count  \
 0    theturnerhouseproject  CKGwWBNBkZq    83             1   
 1    theturnerhouseproject  CKGwWBNBkZq    83             2   
 2    theturnerhouseproject  CKGwWBNBkZq    83             3   
 3    theturnerhouseproject  CKGwWBNBkZq    83             4   
 4    theturnerhouseproject  CKGwWBNBkZq    83             5   
 ..                     ...          ...   ...           ...   
 513       sweatsandthecity  CK1szlYD6jy    92            40   
 514       sweatsandthecity  CK1szlYD6jy    92            41   
 515       sweatsandthecity  CK1szlYD6jy    92            42   
 516       sweatsandthecity  CK1szlYD6jy    92            43   
 517       sweatsandthecity  CK1szlYD6jy    92            44   
 
                                          text_comments  
 0    Rejoining @storyofmyhome for day 16: my precio...  
 1    Looks great, can I ask what the wall to wall d...  
 2     It's such a beautiful

In [171]:
import demoji
#demoji.download_codes() 

####
demoji.findall(final_df_comments['text_comments'][1])

{'😍': 'smiling face with heart-eyes',
 '💪🏼': 'flexed biceps: medium-light skin tone',
 '🤷🏼\u200d♀️': 'woman shrugging: medium-light skin tone'}

In [ ]:
pip install langdetect

from langdetect import detect_langs
detect_langs("INPUT")

In [ ]:
#Introduce empty dataframe to fill!
## comments
df['final_df_comments'+range_name] = pd.DataFrame(columns=['name','link_name','#post', 'comment_count', 'text_comments'])





before_last = []
        before_first = []
        after_last = []
        after_first = []
        c_text_a = []
        c_text_b = []
        while True:
            try:
                #load more
                print('load more...')
                time.sleep(3)
                load_more_comments_element = driver.find_element_by_css_selector('.dCJp8.afkep')
                time.sleep(0.5)
                load_more_comments_element.click()
                time.sleep(3)
                
            
                #Check comments
                soup = BeautifulSoup(driver.page_source,'lxml')
                c = soup.find_all('div',attrs={'class':'C4VMK'})
                soup_2 = BeautifulSoup(str(c),'lxml') 
                spans = soup_2.find_all('span')
                c_text = [i.text.strip() for i in spans if i != '']
                while 'Verified' in c_text:
                    c_text.remove('Verified')
                c_text = c_text[1::2]
                before_last = c_text[-4:-1]
                before_first = c_text[0:3]
                c_text_b = c_text
    
                #Break if last three comments are equal
                if before_last == after_last and before_first == after_first and len(c_text_b) == len(c_text_a):
                    print("Last 3 and First 3 comments are the same")
                    break
                else:
                    print('load more... if there are any')
        
                #Load comments
                try:
                    time.sleep(3)
                    load_more_comments_element = driver.find_element_by_css_selector('.dCJp8.afkep')
                    print('load more...')
                    time.sleep(0.5)
                    load_more_comments_element.click()
                    time.sleep(3)
                except:
                    print('no further comments..')
        
                #Check comments, again!
                soup = BeautifulSoup(driver.page_source,'lxml')
                c = soup.find_all('div',attrs={'class':'C4VMK'})
                soup_2 = BeautifulSoup(str(c),'lxml') 
                spans = soup_2.find_all('span')
                c_text = [i.text.strip() for i in spans if i != '']
                while 'Verified' in c_text:
                    c_text.remove('Verified')
                c_text = c_text[1::2]
                after_last = c_text[-4:-1]
                after_first = c_text[0:3]
                c_text_a = c_text
            
            except:
                print('no further comments..')
                break
            
        #get number of comments
        if len(c_text) > 0:
            c = len(c_text)
            counter_comments = list(accumulate([len([i]) for i in c_text if i != '']))
            name_comments = [n.text] * len(counter_comments)
            post_link_comments = [link[28:-1]] * len(counter_comments)
            print(c, "comments for", link)
            #append #comments [variable]
            comments.append(c)
            #Add counter list to existing list with comment counts
            comment_counter_by_post = counter_comments
            counter_posts_add = counter_posts
        else:
            c = 0
            zero_counter_comments = []
            zero_counter_posts = []
            c_text = []
            name_comments = []
            post_link_comments = []
            print(c, "comments for", link)
            #append #comments [variable]
            comments.append(c)
            #Add counter list to existing list with comment counts
            comment_counter_by_post = zero_counter_comments
            counter_posts_add = zero_counter_posts
    
    
        time.sleep(1)
    
        #get spans of replies
        r = soup.find_all('span',attrs={'class':'EizgU'})
        soup_2 = BeautifulSoup(str(r),'lxml') 
        spans = soup_2.find_all('span')
        r = [i.text.strip() for i in spans if i != '']
    
        #get reply sum and count
        r_sum = 0
        r_count = 0
        for i in r:
            if not re.findall(r'\d+', i):
                r_extracted = ['0']
                r_sum = r_sum + int(r_extracted[0])
            else:
                r_extracted = re.findall(r'\d+', i)[0]
                r_sum = r_sum + int(r_extracted)
                r_count = r_count + 1 
            
        reply_count.append(r_count)
        reply_sum.append(r_sum)
    
        #Print time for comments
        t1_1 = time.time() - t1
        print(source)
        print("Scraping time for post", counter_posts, ": ", t1_1)
        time_vector.append(round(t1_1,1))
    
        ## comments
        df['comments_df'] = {'name':name_comments,
                                        'link_name':post_link_comments,
                                        '#post':counter_posts_add, 
                                        'comment_count':comment_counter_by_post, 
                                        'text_comments':c_text}
        df['comments_df'] = pd.DataFrame(df['comments_df'])
    
        ## Put new post to existing df
        df['final_df_comments'+range_name] = df['final_df_comments'+range_name].append(df['comments_df'], ignore_index=True)
        

### Get Image Labels from Clarifai

In [ ]:
pip install clarifai --upgrade

In [ ]:
import pandas as pd
from clarifai.rest import ClarifaiApp
app = ClarifaiApp(api_key='4dcb4889ac4f473f82643957aa4eb9ba')

from clarifai.rest import ClarifaiApp
urls_input = ['https://scontent-lga3-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/150359929_118535793533904_4094127522305847197_n.jpg?_nc_ht=scontent-lga3-1.cdninstagram.com&_nc_cat=100&_nc_ohc=0X9g3kcH_akAX8TjeDn&tp=1&oh=7f8de3d6f6ba4373e146c08defb22dfa&oe=605364EF',
              'https://scontent-lga3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/148954314_2477315665897906_234750061105564779_n.jpg?_nc_ht=scontent-lga3-1.cdninstagram.com&_nc_cat=110&_nc_ohc=dGAv3sdgOpAAX_3SgLP&tp=1&oh=93ad6cfdd3f9aef302a8fa9e099cdafb&oe=60564DBE',
              'https://scontent-lga3-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/135693622_1059715951199597_8271416798460576238_n.jpg?_nc_ht=scontent-lga3-1.cdninstagram.com&_nc_cat=105&_nc_ohc=VSXxKiMZd9sAX8PeuT-&tp=1&oh=3d7119779915618c28220c60a721f920&oe=6054AA0B']
model = app.public_models.general_model

#define a certainty threshold
prob = 0.9

#define empty lists
labels = []
urls_output = []
label_counts = []
#loop through urls to extract labels
for url in urls_input:
    response = model.predict_by_url(url=url)
    concepts = response['outputs'][0]['data']['concepts']
    initial_df = pd.DataFrame(concepts)[['name','value']]
    label_list = list(initial_df[initial_df['value'] >= prob]['name'])
    labels.append(label_list)
    urls_output.append(response['outputs'][0]['input']['data']['image']['url'])
    label_counts.append(len(label_list))
    
#Build dictionary to subsequently create a dataframe
dictionary = {'labels':labels,
              'urls':urls_output,
              'count':label_counts}
df = pd.DataFrame(dictionary)
df